### COmparison of all models. 
    1. Pick the best parameters of all the models and compute the error metrics for cmparison 

In [33]:
import pandas as pd
import numpy as np 
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, confusion_matrix, accuracy_score
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [9]:
data = pd.read_csv("D:/AP/Data/CrossSell/Ins_train.csv")

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [11]:
data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


## Data Exploration observations 
    1. Entries(rows) : 381109 , cols : input = 10, target = 1 
    2. No missing values
    3. Gender, Vehicle_Age and Vehicle_Damage are categorial variables -- Encoding 
    4. Response is Binay -- Binary classification 
    

In [12]:
data.Vehicle_Age.value_counts()

1-2 Year     200316
< 1 Year     164786
> 2 Years     16007
Name: Vehicle_Age, dtype: int64

In [13]:
data.Vehicle_Damage.value_counts()

Yes    192413
No     188696
Name: Vehicle_Damage, dtype: int64

In [15]:
replaceStruct = {
                "Vehicle_Age":     {"< 1 Year": 1, "1-2 Year": 2 ,"> 2 Years": 3},
                "Gender":     {"Female": 0, "Male": 1 },
                "Vehicle_Damage":     {"No": 0, "Yes": 1 } 
                    }

In [16]:
data = data.replace(replaceStruct)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  int64  
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  int64  
 7   Vehicle_Damage        381109 non-null  int64  
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 34.9 MB


In [21]:
data.columns

Index(['id', 'Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response'],
      dtype='object')

In [22]:
y = data['Response']
X = data[['Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage']]

In [28]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 123)

In [30]:
print('Train size :',x_train.shape)
print('Test size:', x_test.shape)

Train size : (304887, 10)
Test size: (76222, 10)


In [32]:
folds = KFold(n_splits = 10, shuffle = True, random_state = 123)

### Try all emsemble techniques 
1. Random Forest 
2. GBM 
3. XGboost 
4. Lightgbm 

In [34]:
rf = RandomForestClassifier()


## RandomForest 

In [36]:
rf_cv_score = cross_val_score(rf, x_train, y_train, cv = folds, verbose = 1, n_jobs=-1)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  2.4min finished


In [37]:
rf_cv_score

array([0.86513169, 0.86378694, 0.86942832, 0.8673292 , 0.86352455,
       0.86637804, 0.86805077, 0.86729205, 0.86486486, 0.86725925])

In [40]:
print("Mean accuracy:", np.mean(rf_cv_score))
print("Standard deviation", np.std(rf_cv_score))

Mean accuracy: 0.8663045669459928
Standard deviation 0.0018238476364200577


## GBM 

In [42]:
gb = GradientBoostingClassifier()
gb_cv_score = cross_val_score(gb, x_train, y_train, cv = folds, verbose = 1, n_jobs=-1)
print(gb_cv_score)
print("Mean accuracy:", np.mean(gb_cv_score))
print("Standard deviation", np.std(gb_cv_score))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


[0.87533209 0.87454492 0.87697202 0.87890715 0.87618485 0.87838237
 0.87920234 0.87759118 0.87683679 0.87965757]
Mean accuracy: 0.8663045669459928
Standard deviation 0.0018238476364200577


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  2.0min finished


In [43]:
print("GB Mean accuracy:", np.mean(gb_cv_score))
print("GB Standard deviation", np.std(gb_cv_score))

GB Mean accuracy: 0.8773611272443892
GB Standard deviation 0.001612691056527964


In [44]:
xgb = xgb.XGBClassifier()
xgb_cv_score = cross_val_score(xgb, x_train, y_train, cv = folds, verbose = 1, n_jobs=-1)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   39.2s finished


In [46]:
print(xgb_cv_score)
print("XGB Mean accuracy:", np.mean(xgb_cv_score))
print("XGB Standard deviation", np.std(xgb_cv_score))

[0.87536489 0.87454492 0.87703762 0.87893995 0.87621765 0.87841517
 0.87930073 0.87759118 0.87677119 0.87962477]
XGB Mean accuracy: 0.8773808061502262
XGB Standard deviation 0.0016191077256066198


In [48]:
lg = LGBMClassifier(silent=False)
lgb_cv_score = cross_val_score(lg, x_train, y_train, cv = folds, verbose = 1, n_jobs=-1)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    7.2s finished


In [50]:
print(lgb_cv_score)
print("LGB Mean accuracy:", np.mean(lgb_cv_score))
print("LGB Standard deviation", np.std(lgb_cv_score))

[0.8751025  0.87457772 0.87710322 0.87864476 0.87598806 0.87834957
 0.87890715 0.87742718 0.87644319 0.87955917]
LGB Mean accuracy: 0.8772102510070715
LGB Standard deviation 0.001589535992628583


In [52]:
adb = AdaBoostClassifier()
adb_cv_score = cross_val_score(adb, x_train, y_train, cv = folds, verbose = 1, n_jobs=-1)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:   32.1s finished


In [54]:
print(adb_cv_score)
print("ADB Mean accuracy:", np.mean(adb_cv_score))
print("ADB Standard deviation", np.std(adb_cv_score))

[0.87536489 0.87454492 0.87703762 0.87887435 0.87618485 0.87841517
 0.87933353 0.87755838 0.87673839 0.87952637]
ADB Mean accuracy: 0.8773578465123274
ADB Standard deviation 0.00160656114383834
